Задание:
Для любых видео восстановить траекторию движения (t вектор). Выполнить визуализацию. 
Определить параметры которые влияют на "точность" определения вектора t
Использовать решение на базе нейронных сетей. 
Любые идеи. 

***slam прикрутить. 
Видео 1 но что бы работало.    

В данном файле использован код из урока для сравнения с другими способами построения траектории.
Выполняется поиск особых точек детектором SIFT, вычисление ессенциальной матрицы средствами OpenCV.
Из видео выбран каждый 10 кадр, для существенного смещения камеры.

In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import os
import cv2
import numpy as np
from tqdm import tqdm
from mpl_toolkits import mplot3d


In [3]:
import cv2

def filter_matches_distance(matches, dist_threshold):
    """
    Filter matched features from two images by distance between the best matches

    Arguments:
    match -- list of matched features from two images
    dist_threshold -- maximum allowed relative distance between the best matches, (0.0, 1.0) 

    Returns:
    filtered_match -- list of good matches, satisfying the distance threshold
    """
    filtered_match = []
    for m, n in matches:
        if m.distance <= dist_threshold*n.distance:
            filtered_match.append(m)

    return filtered_match

In [4]:
def match_features(des1, des2, matching='BF', detector='sift', sort=True, k=2):
    """
    Match features from two images

    Arguments:
    des1 -- list of the keypoint descriptors in the first image
    des2 -- list of the keypoint descriptors in the second image
    matching -- (str) can be 'BF' for Brute Force or 'FLANN'
    detector -- (str) can be 'sift or 'orb'. Default is 'sift'
    sort -- (bool) whether to sort matches by distance. Default is True
    k -- (int) number of neighbors to match to each feature.

    Returns:
    matches -- list of matched features from two images. Each match[i] is k or less matches for 
               the same query descriptor
    """
    if matching == 'BF':
        if detector == 'sift':
            matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)
        elif detector == 'orb':
            matcher = cv2.BFMatcher_create(cv2.NORM_HAMMING2, crossCheck=False)
        matches = matcher.knnMatch(des1, des2, k=k)
    elif matching == 'FLANN':
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        matcher = cv2.FlannBasedMatcher(index_params, search_params)
        matches = matcher.knnMatch(des1, des2, k=k)
    
    if sort:
        matches = sorted(matches, key = lambda x:x[0].distance)

    return matches

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def create_trajectory(poses):
    trajectory = [np.array([0, 0, 0])]
    current_pose = np.eye(4)
    pose_cam = [np.array([[0, 0, 0],[0, 0, 0],[0, 0, 0]])]

    for R, t in poses:
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = t.T
        current_pose = np.dot(current_pose, T)
        trajectory.append(current_pose[:3, 3])
        pose_cam.append(current_pose[:3, :3])

    return np.array(trajectory),np.array(pose_cam)

In [8]:

# Получить список файлов с изображениями
image_folder = "./frames"
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])

# Получить список файлов с изображениями
image_folder = "./frames"
image_files = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith(".png")])

K = np.array([[3000 ,   0.    , 960],
       [  0.    , 3000 , 540 ],
      [  0.    ,   0.    ,   1.]], dtype=np.float32)

sift = cv2.SIFT_create()

# Сопоставление особых точек (BFMatcher с нормой Hamming)
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

poses = []
# Сохранить позиции камеры
positions = [np.array([0, 0, 0])]

# Итерировать через каждую пару соседних изображений
for i in tqdm(range(len(image_files) - 1)):
    try:
       img1 = cv2.imread(image_files[i], cv2.IMREAD_GRAYSCALE)
       img2 = cv2.imread(image_files[i + 1], cv2.IMREAD_GRAYSCALE)
      #  img1 = cv2.undistort(img1, K, dist, None, K)
      #  img2 = cv2.undistort(img2, K, dist, None, K)

       keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
       keypoints2, descriptors2 = sift.detectAndCompute(img2, None)
      
       
       matches = match_features(descriptors1, descriptors2, matching='BF', detector='sift', sort=True)
       matches = filter_matches_distance(matches, 0.7)
       #matches = matcher.match(np.array(descriptors1,dtype=np.uint8), np.array(descriptors2,dtype=np.uint8))

       pts1 = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
       pts2 = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
       
       F, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.LMEDS, prob=0.999, threshold=1.0)     
       
       a, R, t, b = cv2.recoverPose(F, pts1, pts2, K, mask)
       #print(R)
       poses.append((R, t))

       # Обновить позицию камеры
       current_position = positions[-1]
       new_position = current_position + np.dot(R, t).T[0]
       positions.append(new_position)
    except:
       pass   
trajectory,pose_cam = create_trajectory(poses)
  


100%|██████████| 113/113 [01:37<00:00,  1.16it/s]


In [9]:
import plotly.graph_objects as go
import numpy as np
# Создание фигуры
fig = go.Figure()


fig.add_trace(go.Scatter3d(
    x=trajectory[:, 0],
    y=trajectory[:, 1],
    z=trajectory[:, 2],
    mode='lines+markers',
    marker=dict(size=5, color='blue'),
    line=dict(color='blue', width=2),
    name='Camera Trajectory'
))

# Добавление ориентации камеры в каждой точке траектории
for i, (R, t) in enumerate(zip(pose_cam,trajectory)):
    # Направление камеры (ось Z камеры)
    camera_direction = R @ np.array([0, 0, 1])  # Направление оси Z камеры
    camera_direction_end = t + camera_direction * 0.5  # Конец вектора направления

    # Добавляем линию, представляющую направление камеры
    fig.add_trace(go.Scatter3d(
        x=[t[0], camera_direction_end[0]],
        y=[t[1], camera_direction_end[1]],
        z=[t[2], camera_direction_end[2]],
        mode='lines',
        line=dict(color='green', width=2),
        name=f'Camera Direction {i}' if i == 0 else None,
        showlegend=False if i > 0 else True
    ))

# Настройка макета графика
fig.update_layout(
    title='Camera Motion Trajectory',
    scene=dict(
        xaxis_title='X-axis',
        yaxis_title='Y-axis',
        zaxis_title='Z-axis'
    ),
    showlegend=True
)

# Показать график
fig.show()

Данный способ работает быстро (113 кадров обработаны за 1:37), однако в сравнении с другими способами показывает неточный (даже ложный) результат.